In [0]:
%run "./Silver"

In [0]:
source_path = "abfss://stream-data@ecommerceproject2.dfs.core.windows.net/"
bronze_table_path = "abfss://bronze@ecommerceproject2.dfs.core.windows.net/orders"
delta_table_path="abfss://bronze@ecommerceproject2.dfs.core.windows.net/orders/delta_table/"
silver_table_path = "abfss://silver@ecommerceproject2.dfs.core.windows.net/orders/"

In [0]:
extractor = Order_items()
orderitemDf = extractor.extract()

extractor = Products_Category()
productcatDf = extractor.extract()

extractor = Products()
productDf = extractor.extract()

extractor=Customers()
customerDf = extractor.extract()

extractor=Geolocation()
geoDf=extractor.extract()

extractor=Sellers()
sellerDf=extractor.extract()

extractor=Payments()
paymentDf=extractor.extract()

extractor = StreamingProcessor(source_path, bronze_table_path)
orderDf = extractor.start_streaming_job()
orderDf.awaitTermination(30)

In [0]:
preprocessed_order_items_df = get_preprocessed_order_items(orderitemDf)

preprocessed_products_df = get_preprocessed_products(productDf)

preprocessed_product_category_df = get_preprocessed_product_category(productcatDf)

preprocessed_customer_df = get_preprocessed_customers(customerDf)

preprocessed_geo_df=get_preprocessed_geolocation(geoDf)

preprocessed_seller_df=get_preprocessed_sellers(sellerDf)

preprocessed_payment_df=get_preprocessed_payments(paymentDf)

preprocessed_order_df=IncrementalPreProcessor(delta_table_path, silver_table_path)
query=preprocessed_order_df.preprocess_stream()
query.awaitTermination(30)

preprocessed_order_df=spark.readStream.format("delta").load(silver_table_path)

In [0]:
# orders_join_df = order_items_join(preprocessed_order_df,preprocessed_order_items_df)

# cat_rev_join_df = category_revenue_join(preprocessed_order_df,preprocessed_order_items_df,preprocessed_products_df,preprocessed_product_category_df)

# cust_join_df = customer_spending_join(preprocessed_order_df,preprocessed_order_items_df,preprocessed_customer_df)

# geo_join_df = geographic_revenue_join(preprocessed_order_df,preprocessed_order_items_df,preprocessed_customer_df,preprocessed_geo_df)

# order_seller_df=order_seller_join(preprocessed_order_df,preprocessed_order_items_df,preprocessed_seller_df)

order_payments_df=order_payments_join(preprocessed_order_df,preprocessed_payment_df)

In [0]:
# class WorkFlowRunner:
#     def __init__(self,name):
#         self.name=name
#     def runner(self):
#         if name=="TotalRevenueWorkflow":
#             firstworkFlow = TotalRevenueWorkflow()
#             return firstworkFlow.runner()
#         elif name=="ProductRevenueWorkflow":
#             secondworkFlow = ProductRevenueWorkflow()
#             return secondworkFlow.runner()
#         elif name=="TopSellingWorkflow":
#             thirdworkFlow = TopSellingWorkflow()
#             return thirdworkFlow.runner()
#         elif name=="TimelyRevenueWorkflow":
#             fourthworkFlow = TimelyRevenueWorkflow()
#             return fourthworkFlow.runner()
#         elif name=="GeoRevenueWorkflow":
#             fifthworkFlow = GeoRevenueWorkflow()
#             return fifthworkFlow.runner()
#         elif name=="CustomerValueWorkflow":
#             sixthworkFlow = CustomerValueWorkflow()
#             return sixthworkFlow.runner()
#         elif name=="AOVWorkflow":
#             seventhworkFlow = AOVWorkflow()
#             return seventhworkFlow.runner()
#         elif name=="ReturnRateWorkflow":
#             eighthworkFlow = ReturnRateWorkflow()
#             return eighthworkFlow.runner()
#         elif name=="CustomerRetentionWorkflow":
#             ninthworkFlow = CustomerRetentionWorkflow()
#             return ninthworkFlow.runner()

In [0]:
# class CustomerRetentionWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):
#         Cust_retention = CustomerRetentionTransformer()
#         Cust_retentionTransformerDf = Cust_retention.transform(cust_join_df)
#         display(Cust_retentionTransformerDf)
        
#         query=(Cust_retentionTransformerDf.writeStream
#                .format("delta")
#                .outputMode("complete")
#                .option("checkpointLocation", "abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/cust_retent/checkpoint")
#                .start("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/cust_retent/delta_table"))
#         query.awaitTermination(15)

In [0]:
# name = "CustomerRetentionWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class TotalRevenueWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):
#         TotalRevenue = TotalRevenueTransformer()
#         TotalRevenueTransformerDf = TotalRevenue.transform(preprocessed_order_items_df)
#         display(TotalRevenueTransformerDf)

#         TotalRevenueTransformerDf.write.format("delta").mode("overwrite").save("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/total_revenue/delta_table")

In [0]:
# name = "TotalRevenueWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class ProductRevenueWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):
#         revenue_by_product_transformer = RevenueByProductCategoryTransformer()
#         revenue_result = revenue_by_product_transformer.transform(cat_rev_join_df)
#         display(revenue_result)

#         query=(revenue_result.writeStream
#                .format("delta")
#                .outputMode("complete")
#                .option("mergeSchema", "true")
#                .option("checkpointLocation", "abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/product_revenue/checkpoint")
#                .start("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/product_revenue/delta_table"))
#         query.awaitTermination(15)
#         # spark.sql("create table if not exists eco.gold.product_revenue location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/product_revenue/delta_table'")
#         # gold_df = spark.table("eco.gold.product_revenue")
#         # display(gold_df)

In [0]:
# name = "ProductRevenueWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class TopSellingWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):       
#         top_selling_transformer = TopSellingProduct()
#         top_selling_result = top_selling_transformer.transform(preprocessed_order_items_df)
#         display(top_selling_result)

#         top_selling_result.write.format("delta").mode("overwrite").save("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/top_product/delta_table")

#         # spark.sql("create table if not exists eco.gold.top_product location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/top_product/delta_table'")
        
#         # gold_df = spark.table("eco.gold.top_product")
#         # display(gold_df)        

In [0]:
# name = "TopSellingWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class TimelyRevenueWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):
#         revenue_by_time_transformer = RevenueTrendOverTimeTransformer()
#         revenue_result = revenue_by_time_transformer.transform(orders_join_df)
#         # display(revenue_result)
#         # Start writing the transformation output to another table or sink
#         query = (revenue_result.writeStream
#                  .format("delta")
#                  .outputMode("complete")  # Or "complete" if doing aggregations
#                  .option("checkpointLocation", "abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/revenue_trend/checkpoint")
#                  .start("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/revenue_trend/delta_table"))
#         query.awaitTermination(15)
#         # spark.sql("create table if not exists eco.gold.revenue_trend location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/revenue_trend/delta_table'")
#         # gold_stream_df = spark.readStream.format("delta").load("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/revenue_trend/delta_table")
#         # display(gold_stream_df)

In [0]:
# name = "TimelyRevenueWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class GeoRevenueWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):
#         revenue_by_state_transformer = GeographicRevenueTransformer()
#         revenue_result = revenue_by_state_transformer.transform(geo_join_df)
#         display(revenue_result)
#         query = (revenue_result.writeStream
#                  .format("delta")
#                  .outputMode("complete") 
#                  .option("mergeSchema", "true")
#                  .option("checkpointLocation", "abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/geographic_revenue/checkpoint")
#                  .start("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/geographic_revenue/delta_table"))
#         query.awaitTermination(15)
#         # spark.sql("create table if not exists eco.gold.geographic_revenue location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/geographic_revenue/delta_table'")
#         # gold_stream_df = spark.readStream.format("delta").load("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/geographic_revenue/delta_table")
#         # display(gold_stream_df)
        

In [0]:
# name = "GeoRevenueWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class CustomerValueWorkflow:
#     def __init__(self):
#         pass
#     def runner(self):
#         customer_value_transformer = CustomerValueTransformer()
#         revenue_result = customer_value_transformer.transform(cust_join_df)
#         display(revenue_result)
#         query = (revenue_result.writeStream
#                  .format("delta")
#                  .outputMode("complete")  
#                  .option("checkpointLocation", "abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/customer_value/checkpoint")
#                  .start("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/customer_value/delta_table"))
#         query.awaitTermination(15)
#         # spark.sql("create table if not exists eco.gold.customer_value location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/customer_value/delta_table'")
#         # gold_stream_df = spark.readStream.format("delta").load("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/customer_value/delta_table")
#         # display(gold_stream_df)
        

In [0]:
# name = "CustomerValueWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class ReturnRateWorkflow:
#     def __init__(self):
#         pass 
#     def runner(self):
#         return_rate_transformer = OrderReturnRateTransformer()
#         return_rate_result = return_rate_transformer.transform(preprocessed_order_df)
#         query=(return_rate_result.writeStream
#                .format("delta")
#                .outputMode("complete")
#                .option("mergeSchema", "true")
#                .option("checkpointLocation", "abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/ReturnRate/checkpoint")
#                .start("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/ReturnRate/delta_table"))
#         query.awaitTermination(15)
#         # spark.sql("""
#         #     CREATE TABLE IF NOT EXISTS eco.gold.return_rate 
#         #     LOCATION 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/ReturnRate/delta_table'""")
#         # gold_df = spark.table("eco.gold.return_rate")
#         # display(gold_df)    
#         # return gold_df

In [0]:
# name = "ReturnRateWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()

In [0]:
# class AOVWorkflow:
#     def __init__(self):
#         pass 
#     def runner(self):
#         aov_transformer = AverageOrderValueTransformer()
#         aov_result = aov_transformer.transform(preprocessed_order_items_df)
#         aov_result.write.format("delta").mode("overwrite").save("abfss://gold@ecommerceproject1.dfs.core.windows.net/Result/aov/delta_table")
#         # spark.sql("""
#         #     CREATE TABLE IF NOT EXISTS eco.gold.aov 
#         #     LOCATION 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/aov/delta_table'""")
#         # gold_df = spark.table("eco.gold.aov")
#         # display(gold_df)    
#         # return gold_df

In [0]:
# name = "AOVWorkflow"
# workFlowRunner = WorkFlowRunner(name)
# workFlowRunner.runner()